In [55]:
import matplotlib as mpl
import math 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [56]:
df = pd.read_csv("../Data/Expendatures Teacher Inexperience Out of Cert and Core Index").dropna()

In [57]:
df['log_PER_FED_STATE_LOCAL_EXP'] = np.log(df['PER_FED_STATE_LOCAL_EXP'])

In [58]:
print(df)

      Unnamed: 0                        ENTITY_NAME     ENTITY_CD  \
0              0           MONTESSORI MAGNET SCHOOL   10100010014   
1              1       PINE HILLS ELEMENTARY SCHOOL   10100010016   
2              2          DELAWARE COMMUNITY SCHOOL   10100010018   
3              3     NEW SCOTLAND ELEMENTARY SCHOOL   10100010019   
4              4        ALBANY SCHOOL OF HUMANITIES   10100010023   
...          ...                                ...           ...   
5734        5734  CLYDE-SAVANNAH JUNIOR HIGH SCHOOL  650301040004   
5735        5735            LYONS ELEMENTARY SCHOOL  650501040001   
5736        5736           LYONS SENIOR HIGH SCHOOL  650501040002   
5737        5737                LYONS MIDDLE SCHOOL  650501040003   
5738        5738           MARION ELEMENTARY SCHOOL  650701040001   

      PER_FED_STATE_LOCAL_EXP  PER_TEACH_INEXP  PER_OUT_CERT  ABSENT_RATE  \
0                     13960.0              5.0           0.0         12.4   
1                

In [59]:
features = ['PER_TEACH_INEXP', 'log_PER_FED_STATE_LOCAL_EXP', 'PER_OUT_CERT', 'ABSENT_RATE']
outcome = "CORE_INDEX"
#list(df.columns)

In [60]:
from sklearn.model_selection import train_test_split

model_df = df[features + [outcome]].copy()

df_train, df_temp = train_test_split(model_df, 
                                     train_size=0.8, 
                                     random_state=500, 
                                     shuffle=True
                                    )

# Split the remaining data into validation and test sets
df_val, df_test = train_test_split(df_temp, 
                                   train_size=0.5,  # Assuming you want half of the remaining data for validation and half for testing
                                   random_state=500, 
                                   shuffle=True
                                  )

In [61]:
X_train = df_train[features]
y_train = df_train[outcome]

X_val = df_val[features]
y_val = df_val[outcome]

X_test = df_test[features]
y_test = df_test[outcome]

# Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Create and fit the linear regression model
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

# Now you can use the model to make predictions on the validation or test set
# For example, on the validation set:
predictions_val = linreg.predict(X_val_scaled)

# And on the test set:
predictions_test = linreg.predict(X_test_scaled)

In [62]:
# Assuming 'model' is your trained Linear Regression model
coefficients = linreg.coef_

# Create a DataFrame to display coefficients along with corresponding feature names
coefficients_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})

# Print or display the coefficients
print(coefficients_df)


       Feature  Coefficient
0  ABSENT_RATE   -22.576467


In [63]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming 'model' is your trained Linear Regression model

# Make predictions on the training set
predictions_train = linreg .predict(X_train_scaled)

# Make predictions on the validation set
predictions_val = linreg .predict(X_val_scaled)

# Make predictions on the test set
predictions_test = linreg .predict(X_test_scaled)

# Calculate evaluation metrics for the training set
mae_train = mean_absolute_error(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, predictions_train)

# Calculate evaluation metrics for the validation set
mae_val = mean_absolute_error(y_val, predictions_val)
mse_val = mean_squared_error(y_val, predictions_val)
rmse_val = np.sqrt(mse_val)
r2_val = r2_score(y_val, predictions_val)

# Calculate evaluation metrics for the test set
mae_test = mean_absolute_error(y_test, predictions_test)
mse_test = mean_squared_error(y_test, predictions_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, predictions_test)

# Print the results for each set
print('Training Set Metrics:')
print(f'Mean Absolute Error (MAE): {mae_train}')
print(f'Mean Squared Error (MSE): {mse_train}')
print(f'Root Mean Squared Error (RMSE): {rmse_train}')
print(f'R-squared (R²): {r2_train}')
print()

print('Validation Set Metrics:')
print(f'Mean Absolute Error (MAE): {mae_val}')
print(f'Mean Squared Error (MSE): {mse_val}')
print(f'Root Mean Squared Error (RMSE): {rmse_val}')
print(f'R-squared (R²): {r2_val}')
print()

print('Test Set Metrics:')
print(f'Mean Absolute Error (MAE): {mae_test}')
print(f'Mean Squared Error (MSE): {mse_test}')
print(f'Root Mean Squared Error (RMSE): {rmse_test}')
print(f'R-squared (R²): {r2_test}')


Training Set Metrics:
Mean Absolute Error (MAE): 25.82474528202455
Mean Squared Error (MSE): 1019.7356235512435
Root Mean Squared Error (RMSE): 31.933299603254962
R-squared (R²): 0.33325882239881166

Validation Set Metrics:
Mean Absolute Error (MAE): 25.431890455808286
Mean Squared Error (MSE): 987.4092606052225
Root Mean Squared Error (RMSE): 31.423068924044042
R-squared (R²): 0.3783931204197115

Test Set Metrics:
Mean Absolute Error (MAE): 26.528986843365033
Mean Squared Error (MSE): 1063.131595938293
Root Mean Squared Error (RMSE): 32.60569882609929
R-squared (R²): 0.35988014393892565


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def clean_label(label):
    return label.replace("_", " ").title()

# Assuming df_test is your test dataset
predictions_test = linreg.predict(X_test_scaled)

# Create a DataFrame with actual and predicted values for the test set
scatter_df_test = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predictions_test,
    'log_PER_FED_STATE_LOCAL_EXP': df_test['log_PER_FED_STATE_LOCAL_EXP'],
    'PER_TEACH_INEXP': df_test['PER_TEACH_INEXP'],
    'PER_OUT_CERT': df_test['PER_OUT_CERT'],
    'CORE_INDEX': df_test['CORE_INDEX'], 
    'ABSENT_RATE': df_test['ABSENT_RATE']
})

# Compare predictions on the training set and the test set
for feature in features:
    figure, axes = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
    figure.suptitle(clean_label(feature), fontsize=26)

    # Scatter plot for training set actual and predicted data points
    sns.scatterplot(data=scatter_df_test, x=feature, y='Actual', label='Training Set - Actual', ax=axes[0])
    sns.scatterplot(data=scatter_df_test, x=feature, y='Predicted', label='Training Set - Predicted', ax=axes[0], color="darkorange")

    # Scatter plot for test set actual and predicted data points
    sns.scatterplot(data=scatter_df_test, x=feature, y='Actual', label='Test Set - Actual', ax=axes[1])
    sns.scatterplot(data=scatter_df_test, x=feature, y='Predicted', label='Test Set - Predicted', ax=axes[1], color="darkorange")

    axes[0].set_title("Training Set - Actual vs Predicted")
    axes[1].set_title("Test Set - Actual vs Predicted")

    plt.show()


#### So far, the R-squared looks below 0.5, and scatter plots do not present a clear linear relationship. Thus, we try polynomial regression.

In [ ]:
X_train = df_train[features]
y_train = df_train[outcome]

X_val = df_val[features]
y_val = df_val[outcome]

X_test = df_test[features]
y_test = df_test[outcome]

# Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Create a polynomial regression model (degree = 2, you can change it)
degree = 4
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Fit the model
polyreg.fit(X_train_scaled, y_train)

# Make predictions on the training set
predictions_poly = polyreg.predict(X_train_scaled)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the training set
predictions_train = polyreg.predict(X_train_scaled)

# Make predictions on the validation set
predictions_val = polyreg.predict(X_val_scaled)

# Make predictions on the test set
predictions_test = polyreg.predict(X_test_scaled)

# Calculate evaluation metrics for the training set
mae_train = mean_absolute_error(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, predictions_train)

# Calculate evaluation metrics for the validation set
mae_val = mean_absolute_error(y_val, predictions_val)
mse_val = mean_squared_error(y_val, predictions_val)
rmse_val = np.sqrt(mse_val)
r2_val = r2_score(y_val, predictions_val)

# Calculate evaluation metrics for the test set
mae_test = mean_absolute_error(y_test, predictions_test)
mse_test = mean_squared_error(y_test, predictions_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, predictions_test)

# Print the results for each set
print('Training Set Metrics:')
print(f'Mean Absolute Error (MAE): {mae_train}')
print(f'Mean Squared Error (MSE): {mse_train}')
print(f'Root Mean Squared Error (RMSE): {rmse_train}')
print(f'R-squared (R²): {r2_train}')
print()

print('Validation Set Metrics:')
print(f'Mean Absolute Error (MAE): {mae_val}')
print(f'Mean Squared Error (MSE): {mse_val}')
print(f'Root Mean Squared Error (RMSE): {rmse_val}')
print(f'R-squared (R²): {r2_val}')
print()

print('Test Set Metrics:')
print(f'Mean Absolute Error (MAE): {mae_test}')
print(f'Mean Squared Error (MSE): {mse_test}')
print(f'Root Mean Squared Error (RMSE): {rmse_test}')
print(f'R-squared (R²): {r2_test}')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def clean_label(label):
    return label.replace("_", " ").title()

# Assuming df_test is your test dataset
predictions_test = polyreg.predict(X_test_scaled)

# Create a DataFrame with actual and predicted values for the test set
scatter_df_test = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predictions_test,
    'log_PER_FED_STATE_LOCAL_EXP': df_test['log_PER_FED_STATE_LOCAL_EXP'],
    'PER_TEACH_INEXP': df_test['PER_TEACH_INEXP'],
    'PER_OUT_CERT': df_test['PER_OUT_CERT'],
    'CORE_INDEX': df_test['CORE_INDEX'], 
    'ABSENT_RATE': df_test['ABSENT_RATE']
})

# Compare predictions on the training set and the test set
for feature in features:
    figure, axes = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
    figure.suptitle(clean_label(feature), fontsize=26)

    # Scatter plot for training set actual and predicted data points
    sns.scatterplot(data=scatter_df_test, x=feature, y='Actual', label='Training Set - Actual', ax=axes[0])
    sns.scatterplot(data=scatter_df_test, x=feature, y='Predicted', label='Training Set - Predicted', ax=axes[0], color="darkorange")

    # Scatter plot for test set actual and predicted data points
    sns.scatterplot(data=scatter_df_test, x=feature, y='Actual', label='Test Set - Actual', ax=axes[1])
    sns.scatterplot(data=scatter_df_test, x=feature, y='Predicted', label='Test Set - Predicted', ax=axes[1], color="darkorange")

    axes[0].set_title("Training Set - Actual vs Predicted")
    axes[1].set_title("Test Set - Actual vs Predicted")

    plt.show()


In [ ]:
### Model Comparison

from sklearn.metrics import mean_squared_error, r2_score

linear_predictions = linreg.predict(X_val_scaled)
poly_predictions = polyreg.predict(X_val_scaled)

linear_mse = mean_squared_error(y_val, linear_predictions)
linear_r2 = r2_score(y_val, linear_predictions)

poly_mse = mean_squared_error(y_val, poly_predictions)
poly_r2 = r2_score(y_val, poly_predictions)

print("Linear Model:")
print(f"MSE: {linear_mse}")
print(f"R-squared: {linear_r2}")
print()

print("Polynomial Model:")
print(f"MSE: {poly_mse}")
print(f"R-squared: {poly_r2}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.scatter(y_val, linear_predictions, label='Linear Model', color='blue')
plt.scatter(y_val, poly_predictions, label='Polynomial Model', color='orange')

plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle='--', color='gray', label='Ideal Fit')

plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Model Comparison: Linear vs. Polynomial')
plt.legend()
plt.show()


In [ ]:
### Trying something new

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
params_rf = {
    'n_estimators': 100,  # adjust as needed
    'max_depth': None,  # or specify a value
    'min_samples_split': 2,  # adjust as needed
    'min_samples_leaf': 1,  # adjust as needed
    'max_features': 'auto',  # adjust as needed
    'random_state': 42  # set a seed for reproducibility
}


In [ ]:
rf_model = RandomForestRegressor(**params_rf)
rf_model.fit(X_train_scaled, y_train)


In [ ]:
# Assuming X_val_scaled is your validation data
rf_predictions = rf_model.predict(X_val_scaled)


In [ ]:
rf_mse = mean_squared_error(y_val, rf_predictions)
rf_r2 = r2_score(y_val, rf_predictions)

print("Random Forest Model:")
print(f"MSE: {rf_mse}")
print(f"R-squared: {rf_r2}")


In [ ]:
plt.figure(figsize=(10, 6))

plt.scatter(y_val, rf_predictions, label='Random Forest Model', color='purple')

plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle='--', color='gray', label='Ideal Fit')

plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Random Forest Model Prediction')
plt.legend()
plt.show()
